In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

# Proxy configuration
PROXY = "127.0.0.1:24000"
# Path to chromedriver executable
PATH = "/opt/homebrew/Caskroom/chromedriver/121.0.6167.85/chromedriver-mac-arm64/chromedriver"

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--proxy-server=%s' % PROXY)

# Initialize Chrome WebDriver with the specified path and options
service = Service(PATH)
# chrome = webdriver.Chrome(service=service, options=chrome_options)

player_ids = ['677950', '572233', '682998', '666150', '656896', '553993', '672515', '672695', '607054', '664983', '645444', '606466', '666971', '656976']
timeout = 25
mlb_seasons = ['2023']

In [2]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

In [3]:
url = r"https://baseballsavant.mlb.com/visuals/statcast-pitch-highlighter?playerId=502671&type=&batHand=&pitchHand=&pitchType=&year=2023&zone="
driver.get(url)

element_present = EC.presence_of_element_located((By.XPATH, r'//*[@id="ddlPitchTypes"]'))
WebDriverWait(driver, timeout).until(element_present)

headers_present = EC.presence_of_element_located((By.XPATH, r'//*[@id="calculations"]/table/thead/tr'))
WebDriverWait(driver, timeout).until(headers_present)

select = Select(driver.find_element(By.XPATH, r'//*[@id="ddlPitchTypes"]'))
select.select_by_index(0)
pitchType = select.first_selected_option.text
src = driver.page_source
parser = BeautifulSoup(src, "lxml")
table = parser.find("div", attrs = {"id": "calculations"})
headers = table.findAll('th')
headerlist = [h.text.strip() for h in headers[:]]
headerlist[0] = "Name"

stats = pd.DataFrame(columns=headerlist)

driver.quit()

In [4]:
player_ids = ['677950']

In [5]:
player_ids

['677950']

In [6]:
for player in player_ids:
    for season in mlb_seasons:
        for i in range(len(select.options)):
            url = f"https://baseballsavant.mlb.com/visuals/statcast-pitch-highlighter?playerId={player}&type=&batHand=&pitchHand=&pitchType=&year={season}&zone="
            driver.get(url)
        
            element_present = EC.presence_of_element_located((By.XPATH, r'//*[@id="ddlPitchTypes"]'))
            WebDriverWait(driver, timeout).until(element_present)

            data_present = EC.presence_of_element_located((By.XPATH, r'//*[@id="calculations-tr_0"]'))
            WebDriverWait(driver, timeout).until(data_present)
    
            select = Select(driver.find_element(By.XPATH, r'//*[@id="ddlPitchTypes"]'))
            select.select_by_index(f"{i}")
            pitchType = select.first_selected_option.text
            src = driver.page_source
            parser = BeautifulSoup(src, "lxml")
            table = parser.find("div", attrs = {"id": "calculations"})
            headers = table.findAll('th')
            headerlist = [h.text.strip() for h in headers[:]]
            headerlist[0] = "Name"
            rows = table.findAll('tr')[1:]
            player_stats = [[td.getText().strip() for td in rows[i].findAll('td')] for i in range(len(rows))]
            if player_stats[0][4] == "": player_stats[0][4] = pitchType
            if player_stats[0][1] == "": player_stats[0][1] = season

            stats2 = pd.DataFrame(player_stats, columns=headerlist)
            stats = pd.concat([stats,stats2], ignore_index=True)

In [7]:
stats

,Name,Season,Total Pitches,Pitches,Pitch Types,%,PA,AB,H,1B,...,BA,SO,Swings,Misses,Sw%,Whiff%,K%,wOBA,EV,Spin Rate
0,Thomas,2023,1489,1489,All Pitches,100.0,401,374,86,55,...,.230,86,731,183,49.1,25.0,21.4,.278,88.3,2244
1,Thomas,2023,1489,757,Fastballs,50.8,197,182,49,28,...,.269,32,366,79,48.3,21.6,16.2,.339,90.0,2252
2,Thomas,2023,1489,245,Offspeed,16.5,76,72,13,9,...,.181,17,123,32,50.2,26.0,22.4,.203,85.6,1677
3,Thomas,2023,1489,487,Breaking,32.7,128,120,24,18,...,.200,37,242,72,49.7,29.8,28.9,.229,87.1,2515
4,Thomas,2023,1489,198,Changeup,13.3,60,56,10,8,...,.179,12,97,25,49.0,25.8,20.0,.188,84.2,1752
5,Thomas,2023,1489,177,Curveball,11.9,42,39,3,3,...,.077,13,84,25,47.5,29.8,31.0,.096,87.6,2618
6,Thomas,2023,1489,0,Eephus,0.0,0,0,0,0,...,--,0,0,0,--,--,--,--,--,--
7,Thomas,2023,1489,0,BP Fastball,0.0,0,0,0,0,...,--,0,0,0,--,--,--,--,--,--
8,Thomas,2023,1489,119,Cutter,8.0,34,32,10,4,...,.313,3,63,14,52.9,22.2,8.8,.441,87.3,2359
9,Thomas,2023,1489,447,4-Seam Fastball,30.0,109,97,30,18,...,.309,18,215,47,48.1,21.9,16.5,.377,90.6,2275
